In [1]:
import os

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [2]:
import langchain
langchain.verbose = True

In [3]:
# Add OpenAI library
import openai

In [4]:
# Get Configuration Settings
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# Configure OpenAI API using Azure OpenAI
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("ENDPOINT")
openai.api_type = "azure"  # Necessary for using the OpenAI library with Azure OpenAI
openai.api_version = "2024-02-01"  # Latest / target version of the API

## Initial tests without a Cloud Vector Store

In [6]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader

In [7]:
def load_and_process_pdfs(txt_folder_path):
    documents = []
    for file in os.listdir(txt_folder_path):
        if file.endswith('.txt'):
            pdf_path = os.path.join(txt_folder_path, file)
            loader = TextLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
    splits = text_splitter.split_documents(documents)
    return splits

txt_folder_path =  "/Users/mac/Documents/Gospel-Companion/data/Senior/"
splits = load_and_process_pdfs(txt_folder_path)

In [8]:
splits[0:10]

[Document(page_content='Bible Text: [JUG:5:1-31].\n\nLesson 191 - Senior\n\nMemory Verse: [JUG:5:1-31].\n\nCross References"I, even I, will sing unto the LORD; I will sing praise to the LORD God of Israel" (Judges 5:3).\n\nI The Preamble to Deborah and Barak\'s Song\n\n1. Praise is given to God for His deliverance, [JUG:5:1-3].', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson191_Senior.txt'}),
 Document(page_content="1. Praise is given to God for His deliverance, [JUG:5:1-3].\n\n2. God's glory, majesty, and power, manifested in Israel's behalf, is recounted, [JUG:5:4-5]; [DEU:33:2]; [PS:68:7-8]; [HAB:3:3-4].\n\n3. Israel's shameful condition, because of idolatry, is stated, [JUG:5:6-8].", metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson191_Senior.txt'}),
 Document(page_content='4. Commendation is given the faithful governors of Israel, [JUG:5:9].\n\n5. Israel\'s future of peace, righteousness, and just government is revealed -

In [9]:
# SOURCE_FILE_NAME = "/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt"

In [10]:
# loader = TextLoader(SOURCE_FILE_NAME)
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=150, chunk_overlap=50)
# docs = text_splitter.split_documents(documents)

In [11]:
# docs

In [12]:
# OpenAI Settings
model_deployment = "text-embedding-ada-002"
# SDK calls this "engine", but naming it "deployment_name" for clarity

model_name = "text-embedding-ada-002"

In [13]:
openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_version = os.getenv("OPENAI_API_VERSION"), openai_api_key = os.getenv("API_KEY"),
    openai_api_base = os.getenv("ENDPOINT"), openai_api_type = "azure"
)

/Users/mac/Documents/Gospel-Companion/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
db = DocArrayInMemorySearch.from_documents(
    splits, 
    openai_embeddings
)

/Users/mac/Documents/Gospel-Companion/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
Retrying langchain_community.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).
Retrying langchain_community.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-03-15-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain_community.embeddings.o

In [15]:
query = "What are the steps to creation?"
docs = db.similarity_search(query)

In [16]:
docs

[Document(page_content='Steps of Creation', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt'}),
 Document(page_content='1. Why were all the other steps in creation taken before the creation of man?\n\n2. What is said about the creation of man, which distinguishes and separates him from the other creatures?', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson2_Senior.txt'}),
 Document(page_content="The steps of creation day by day are most wonderful. God's great purpose was to provide this world for man. In order that man should be sustained here in life, it was necessary that the earth be prepared for the creating of life. The successive creative acts reveal great wisdom and order. God first", metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt'}),
 Document(page_content='Preparation', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson164_Senior.txt'})]

In [17]:
retriever = db.as_retriever()

In [30]:
llm = ChatOpenAI(temperature = 0.6, openai_api_key = os.getenv("API_KEY"), openai_api_base = os.getenv("ENDPOINT"), model_name="gpt-35-turbo", engine="Voicetask")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


## Usnig ConversationBufferMemory

In [31]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationBufferWindowMemory
from langchain import PromptTemplate

In [32]:
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

In [33]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
            k = 2,
            memory_key="history",
            input_key="question"),
            }
    )

In [45]:
qa_stuff.memory('history')
# load_memory_variables({})["history"]

TypeError: 'NoneType' object is not callable

In [47]:
prompt

AttributeError: 'PromptTemplate' object has no attribute 'context'

In [35]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
------
<ctx>
Steps of Creation

The steps of creation day by day are most wonderful. God's great purpose was to provide this world for man. In order that man should be sustained here in life, it was necessary that the earth be prepared for the creating of life. The successive creative acts reveal great wisdom and order. God first

6. Name what He created each successive day.

7. What was God's comment as He surveyed His completed work?

8. What school of so-called scientists denies in these days the Biblical account of the creation?


In [41]:
qa_stuff.combine_documents_chain.memory
# print(response)

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='What are the steps to creation? Answer in bullet form using the days and the creation'), AIMessage(content="1. Day 1: God created light and separated it from darkness.\n2. Day 2: God created the sky and separated the waters.\n3. Day 3: God created dry land and plants.\n4. Day 4: God created the sun, moon, and stars.\n5. Day 5: God created sea creatures and birds.\n6. Day 6: God created land animals and humans.\n7. God's comment after surveying His completed work was that it was good.\n8. Some members of the scientific community deny the Biblical account of creation.")]), input_key='question', k=2)

In [42]:
retriever

VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x10f5de850>)

In [38]:
query = "Can you give me bible verses that talk in detail about the creation?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
------
<ctx>
While no declaration that God makes needs to be substantiated by evidence or argument, yet other portions of His wonderful Book, touching upon the subject of creation, serve greatly to strengthen one's faith in what here is declared -- as, for example, the words of the Psalmist: "The heavens declare the glory of God; and the firmament sheweth his handiwork. Day unto day uttereth speech, and night unto night sheweth knowledge. There is no speech nor language, where their voice is not heard" [PS:19:1-3]). The heavens, and 

In [32]:
query = "What do those verses say? Write it out"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
------
<ctx>
1. Who wrote the Book of Genesis?

2. What other Books of the Bible did he write?

3. What is the meaning of the name Genesis?

Cross References"Through faith we understand that the worlds were framed by the word of God, so that things which are seen were not made of things which do appear"  (Hebrews 11:3).

The first verse of the inspired Word declares, "In the beginning God created the heaven and the earth." These words thus pronounce God the Creator of the universe; and for him who believes the Bible this statement se

"Sorry, there are no specific verses mentioned in the user's question. Can you please provide more context or clarify your question?"

In [40]:
query = "The verses about the creation you gave out, can you give me what they are in KJV."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
------
<ctx>
The first verse of the inspired Word declares, "In the beginning God created the heaven and the earth." These words thus pronounce God the Creator of the universe; and for him who believes the Bible this statement settles the issue -- God is the Creator of the universe and all that is in it.

While no declaration that God makes needs to be substantiated by evidence or argument, yet other portions of His wonderful Book, touching upon the subject of creation, serve greatly to strengthen one's faith in what here is declared

In [42]:
query = "Can you tell me what is in Genesis 1:1-31 according to KJV. Write it out in verses."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
------
<ctx>
Bible Text: [GEN:1:1-25].

Lesson 1 - Senior

Memory Verse: [GEN:1:1-25].

1. Who wrote the Book of Genesis?

2. What other Books of the Bible did he write?

3. What is the meaning of the name Genesis?

3. What is the meaning of the name Genesis?

4. What is the opening account of this Book?

5. How many days did God devote to the work of creation?

The first verse of the inspired Word declares, "In the beginning God created the heaven and the earth." These words thus pronounce God the Creator of the universe; and for hi

In [ ]:
qa_stuff.combine_documents_chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Can you give me bible verses that talk in detail about the creation?'), AIMessage(content='Yes, Psalm 19:1-3 and Genesis 1:1-31 are two examples of Bible verses that talk about the creation in detail. However, if you\'re looking for a more comprehensive list, I suggest doing a search for "Bible verses about creation" online or in a Bible concordance.')]), input_key='question')